In [1]:
# 개발환경(OS) : colab(Linux)

In [2]:
# python==3.7.12
# albumentations==1.1.0
# numpy==1.19.5
# pandas==1.3.5
# cv2==4.1.2
# sklearn==1.0.2
# json==2.0.9
# torch==1.10.0+cu111
# timm==0.5.4
# transformers==4.16.2

In [3]:
# pakage

!pip uninstall opencv-python-headless==4.5.5.62 --yes
!pip install opencv-python-headless==4.1.2.30

import pickle
import gc
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import random
import os
import json 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split

# timm : 0.5.4
!pip install timm
# albumtations : 1.1.0
!pip install -U albumentations
# transformers : 4.16.2
!pip install transformers

import timm
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup

os.environ["CUDA_VISIBLE_DEVICES"]="0"

Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64


ERROR: Could not find a version that satisfies the requirement opencv-python-headless==4.1.2.30 (from versions: 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.14.51, 3.4.14.53, 3.4.15.55, 3.4.16.59, 3.4.17.61, 3.4.17.63, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.2.52, 4.5.2.54, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64)
ERROR: No matching distribution found for opencv-python-headless==4.1.2.30


  Using cached opencv_python_headless-4.5.5.64-cp36-abi3-win_amd64.whl (35.3 MB)


In [4]:
# get sequence feature information

csv_feature_dict = {'내부 습도 1 최고': [25.9, 100.0],
                    '내부 습도 1 최저': [0.0, 100.0],
                    '내부 습도 1 평균': [23.7, 100.0],
                    '내부 온도 1 최고': [3.4, 47.6],
                    '내부 온도 1 최저': [3.3, 47.0],
                    '내부 온도 1 평균': [3.4, 47.3],
                    '내부 이슬점 최고': [0.2, 34.7],
                    '내부 이슬점 최저': [0.0, 34.4],
                    '내부 이슬점 평균': [0.1, 34.5]}

In [5]:
# label encoder, decoder 

file_path = 'C:/Users/GunhaKim/Desktop/projects/artificial_intelligence/term_project/data'

train_json = sorted(glob(f'{file_path}/train/*/*.json'))

labels = []
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}" #label = crop_disease_risk
        labels.append(label) # labels = [label1, lable2, lable3...]

label_encoder = sorted(np.unique(labels)) # label_encoder : label 종류별로 나열(중복 없음)
label_encoder = {key:value for key,value in zip(label_encoder, range(len(label_encoder)))} # label_encoder를 dictionary 형태로
label_decoder = {val:key for key, val in label_encoder.items()}

In [6]:
# hyper parameters

opt = dict()

opt['batch_size'] = 16
opt['class_n'] = len(label_encoder)
opt['lr'] = 2e-4
opt['embedding_dim'] = 512
opt['feature_n'] = len(csv_feature_dict)
opt['max_len'] = 300
opt['dropout_rate'] = 0.3
opt['epoch_n'] = 25
opt['vision_pretrain'] = True
opt['worker_n'] = 8
opt['folder'] = 'model_weights'
opt['bidirectional'] = True
opt['minmax_dict'] = csv_feature_dict
opt['label_dict'] = label_encoder
opt['enc_name'] = 'resnext50_32x4d'
opt['enc_dim'] = 2048
opt['dec_dim'] = 1024
opt['img_size1'] = 384
opt['img_size2'] = 384
opt['precision'] = 'amp'
opt['seed'] = 42
opt['mix'] = 'cutmix'
opt['mix_prob'] = 0.3
opt['mean'] = [0.485, 0.456, 0.406]
opt['std'] = [0.229, 0.224, 0.225]


device = torch.device("cuda:0")

In [7]:
# fix seed

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(opt['seed'])

In [8]:
# cutmix function

def rand_bbox(size, lam):
    W = size[2] # width
    H = size[3] # height
    cut_rat = np.sqrt(1. - lam) # 자르는 비율, lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4)
    cut_w = np.int(W * cut_rat) # 잘라내는 이미지의 너비
    cut_h = np.int(H * cut_rat) # 잘라내는 이미지의 높이

    # uniform
    cx = np.random.randint(W) # 잘라내는 이미지 위치 x좌표
    cy = np.random.randint(H) # 잘라내는 이미지 위치 y좌표

    #잘라내는 이미지의 4 꼭지점 좌표
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix(data, target, alpha):
    indices = torch.randperm(data.size(0)) # randperm(n) : 0부터 n-1까지의 정수를 무작위로 나열
    shuffled_data = data[indices] # indices를 이용해 data 랜덤한 순서로 배열
    shuffled_target = target[indices] # indices를 이용해 target 랜덤한 순서로 배열

    lam = np.clip(np.random.beta(alpha, alpha),0.3,0.4) # cutting ration를 0.3~0.4 사이로 함
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    # new_data는 순서대로 배열되어 있는 이미지들인데 indices 순서로 배열된 이미지들과 섞고 있음
    new_data[:, :, bby1:bby2, bbx1:bbx2] = data[indices, :, bby1:bby2, bbx1:bbx2] 
    # adjust lambda to exactly match pixel ratio(해당 이미지에서 정확히 차지하는 비율 계산)
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))
    targets = (target, shuffled_target, lam)

    return new_data, targets

In [9]:
# get data

train = sorted(glob(f'{file_path}/train/*'))
test = sorted(glob(f'{file_path}/test/*'))
labelsss = pd.read_csv(f'{file_path}/train.csv')['label']

In [10]:
# k-fold cross validation

skf = StratifiedKFold(n_splits = 5)
train = shuffle(train, random_state=opt['seed'])

folds = []
for idx, (train_idx, val_idx) in enumerate(skf.split(train, labelsss)):
    folds.append((train_idx, val_idx))

In [11]:
# dataset

def trainTransform():
  return Compose([
                  #Transpose(p=0.5),
                  #HorizontalFlip(p=0.5),
                  VerticalFlip(p=0.5),
                  ShiftScaleRotate(p=0.5),
                  RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
                  ], p=1.)
  
def valTransform():
  return Compose([
                  Resize(opt['img_size1'], opt['img_size2']),
                  Normalize(mean=opt['mean'], std=opt['std'], max_pixel_value=255.0, p=1.0),
                  ToTensorV2(p=1.0),
              ], p=1.)

class CustomDataset(Dataset):
    def __init__(self, opt, files, mode, transforms):
        self.files = files
        self.mode = mode
        self.transforms = transforms
        self.csv_check = [0]*len(self.files)
        self.seq = [None]*len(self.files)
        self.minmax_dict = opt['minmax_dict']
        self.max_len = opt['max_len']
        self.label_encoder = opt['label_dict']

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split('\\')[-1]
        
        if self.csv_check[i] == 0:
            csv_path = f'{file}/{file_name}.csv'
            df = pd.read_csv(csv_path)
            

            try:
                estiTime1, estiTime2 = df.iloc[0]['측정시각'], df.iloc[1]['측정시각']
            except:
                estiTime1, estiTime2 = 0, 1

            df = df[self.minmax_dict.keys()]
            df = df.replace('-', 0)
            
            if self.mode == 'train':
                decision = np.random.rand()
                if estiTime1==estiTime2 and len(df)>400:   # 데이터 길이가 길 때 랜덤으로 절반 뽑아오는 작업인듯?
                    if decision > 0.5:
                        df = df[0::2].reset_index(drop=True)
                    else:
                        df = df[1::2].reset_index(drop=True)
            else: 
                if estiTime1==estiTime2 and len(df)>400:
                    df = df[0::2].reset_index(drop=True)
                
            
            # minmax-scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.minmax_dict[col][0]
                df[col] = df[col] / (self.minmax_dict[col][1]-self.minmax_dict[col][0])

            # zero-padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]

            # transpose-to-sequential-data
            seq = torch.tensor(pad, dtype=torch.float32)
            self.seq[i] = seq
            self.csv_check[i] = 1
        else:
            seq = self.seq[i]
        
        # image-transform
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.uint8)              
        img = self.transforms(image=img)['image'] 

        
        if self.mode == 'train' or self.mode == 'val':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = torch.tensor(self.label_encoder[f'{crop}_{disease}_{risk}'], dtype=torch.long)
            
            return img, seq, label
        else:
            return img, seq

In [12]:
# model

class Encoder(nn.Module):
    def __init__(self, opt):
        super(Encoder, self).__init__()
        # resnet 50사용
        self.model = timm.create_model(model_name=opt['enc_name'], 
                                       pretrained=opt['vision_pretrain'], 
                                       num_classes=0)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output


class Decoder(nn.Module):
    def __init__(self, opt):
        super(Decoder, self).__init__()
        self.decoder = nn.GRU(opt['feature_n'], opt['embedding_dim'], bidirectional = opt['bidirectional'])
        self.dense = nn.Linear(2*opt['max_len']*opt['embedding_dim'], opt['dec_dim'])
        
        self.f1 = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['enc_dim']+opt['dec_dim'])
        self.out = nn.Linear(opt['enc_dim']+opt['dec_dim'], opt['class_n'])
        self.dropout = nn.Dropout(opt['dropout_rate'])
        self.relu = nn.ReLU()

    def init_weight(self):
        torch.nn.init.xavier_uniform_(self.f1.weight)  
        torch.nn.init.xavier_uniform_(self.dense.weight)  
        torch.nn.init.xavier_uniform_(self.out.weight)  


    def forward(self, enc_out, dec_inp):
        # csv파일 해석에는 GRU사용
        dec_out, _ = self.decoder(dec_inp)
        dec_out = self.dense(dec_out.view(dec_out.size(0), -1))
        
        # encoder와 decoder로부터 나온 torch데이터를 cocat해서 fc로 돌림
        concat = torch.cat([enc_out, dec_out], dim=1) 
        concat = self.f1(self.relu(concat))
        concat = self.dropout(self.relu(concat))
        output = self.out(concat)
        return output


class CustomModel(nn.Module):
    def __init__(self, opt):
        super(CustomModel, self).__init__()
        self.encoder = Encoder(opt)
        self.decoder = Decoder(opt)
        self.to(device)
        
    def forward(self, img, seq):
        enc_out = self.encoder(img)
        output = self.decoder(enc_out, seq)
        
        return output

In [13]:
# train function

class CustomTrainer:
    def __init__(self, model, folder, fold):
        self.model=model
        
        self.save_dir = f'/content/{folder}'
        if not os.path.exists(self.save_dir):
          os.makedirs(self.save_dir)
          
        self.optimizer = AdamW(model.parameters(), lr=opt['lr'])
        self.scaler = torch.cuda.amp.GradScaler() 

        total_steps = int(len(train_dataset)*opt['epoch_n']/(opt['batch_size']))
        warmup_steps = 1149
        print('total_steps: ', total_steps)
        print('warmup_steps: ', warmup_steps)
        self.scheduler = get_cosine_schedule_with_warmup(self.optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
        self.loss_fn = nn.CrossEntropyLoss()
        self.val_loss_fn = nn.CrossEntropyLoss()

        self.best_score = 0.0


    def run(self, train_loader, val_loader):
        for epoch in range(opt['epoch_n']):
            gc.collect()
            learning_rate = self.optimizer.param_groups[0]['lr']
            print('learning_rate: ', learning_rate)
            print(f'----- train, epoch{epoch+1} -----')
            train_loss, train_score = self.train_function(train_loader, epoch)
            print(' ')
            print(f'train_loss: {train_loss:.6f}, train_score: {train_score:.6f}')

            print('----------------------------------')

            print(f'----- val, epoch{epoch+1} -----')
            with torch.no_grad():
                val_loss, val_score = self.val_function(val_loader)
            print(' ')
            print(f'val_loss: {val_loss:.6f}, val_score: {val_score:.6f}')


            if epoch+1 >= 16 and val_score >= self.best_score:
                torch.save(self.model.state_dict(), self.save_dir+f"/best-acc-epoch{epoch+1}.bin")
                self.best_score=val_score
                print(f'model is saved when epoch is : {epoch+1}')


    def train_function(self, train_loader, epoch):
        self.model.train()

        total_loss = 0.0
        total_score = 0.0
        for bi, data in enumerate(tqdm(train_loader)):
            data = [x.to(device) for x in data]
            img, seq, label = data

            self.optimizer.zero_grad()
            
            # use mix or not
            if opt['mix']!=None and epoch < opt['epoch_n']-10: 
                mix_decision = np.random.rand()
                if opt['mix'] == 'cutmix' and mix_decision < opt['mix_prob']:
                    img, mix_labels = cutmix(img, label, 1.0)
                else: 
                  pass
            else: mix_decision = 1

            if opt['epoch_n']-10 <= epoch:
                assert mix_decision == 1
            
            # use amp or not
            if opt['precision'] == 'float':
                out = self.model(img, seq)

                if mix_decision < opt['mix_prob']:
                    loss = self.loss_fn(out, mix_labels[0])*mix_labels[2] + self.loss_fn(out, mix_labels[1])*(1-mix_labels[2])
                else:
                    loss = self.loss_fn(out, label)

                loss.backward()
                self.optimizer.step()
            else: 
                with torch.cuda.amp.autocast():
                    out = self.model(img, seq)
                    if mix_decision < opt['mix_prob']:
                        loss = self.loss_fn(out, mix_labels[0])*mix_labels[2] + self.loss_fn(out, mix_labels[1])*(1-mix_labels[2])
                    else:
                        loss = self.loss_fn(out, label)

                self.scaler.scale(loss).backward()  
                self.scaler.step(self.optimizer) 
                self.scaler.update()              
            
            self.scheduler.step()
            total_loss+=loss.detach().cpu()

            total_score+=f1_score(label.cpu(), out.argmax(1).cpu(), average='macro')
        return total_loss/len(train_loader), total_score/len(train_loader)

    def val_function(self, val_loader):
        self.model.eval()

        total_loss = 0.0
        preds, targets = [], []
        for bi, data in enumerate(tqdm(val_loader)):
            data = [x.to(device) for x in data]
            img, seq, label = data

            out = self.model(img, seq)
            loss = self.val_loss_fn(out, label)

            total_loss+=loss.detach().cpu()

            pred = out.argmax(1).detach().cpu().tolist()
            target = label.reshape(-1).detach().cpu().tolist()

            preds.extend(pred)
            targets.extend(target)
        
        score = f1_score(targets, preds, average='macro')
        return total_loss/len(val_loader), score

    def log(self, message):
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')

In [14]:
# run

print(opt)
for i in range(len(folds)):
    train_idx, val_idx = folds[i]
    
    print(f'{i+1}th fold training is start')
    
    # data
    train_dataset = CustomDataset(opt, np.array(train)[train_idx], mode='train', transforms=trainTransform())
    val_dataset = CustomDataset(opt, np.array(train)[val_idx], mode='val', transforms=valTransform())
    print('num of train: ', len(train_dataset))
    print('num of val: ', len(val_dataset))

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=opt['batch_size'], shuffle=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2*opt['batch_size'], shuffle=False)

    # model
    custom_model = CustomModel(opt)

    # trainer
    custom_trainer = CustomTrainer(model=custom_model, folder=opt['folder']+f'/fold{i+1}', fold=i+1)
    custom_trainer.run(train_dataloader, val_dataloader)

{'batch_size': 2, 'class_n': 25, 'lr': 0.0002, 'embedding_dim': 512, 'feature_n': 9, 'max_len': 300, 'dropout_rate': 0.3, 'epoch_n': 25, 'vision_pretrain': True, 'worker_n': 8, 'folder': 'model_weights', 'bidirectional': True, 'minmax_dict': {'내부 습도 1 최고': [25.9, 100.0], '내부 습도 1 최저': [0.0, 100.0], '내부 습도 1 평균': [23.7, 100.0], '내부 온도 1 최고': [3.4, 47.6], '내부 온도 1 최저': [3.3, 47.0], '내부 온도 1 평균': [3.4, 47.3], '내부 이슬점 최고': [0.2, 34.7], '내부 이슬점 최저': [0.0, 34.4], '내부 이슬점 평균': [0.1, 34.5]}, 'label_dict': {'1_00_0': 0, '2_00_0': 1, '2_a5_2': 2, '3_00_0': 3, '3_a9_1': 4, '3_a9_2': 5, '3_a9_3': 6, '3_b3_1': 7, '3_b6_1': 8, '3_b7_1': 9, '3_b8_1': 10, '4_00_0': 11, '5_00_0': 12, '5_a7_2': 13, '5_b6_1': 14, '5_b7_1': 15, '5_b8_1': 16, '6_00_0': 17, '6_a11_1': 18, '6_a11_2': 19, '6_a12_1': 20, '6_a12_2': 21, '6_b4_1': 22, '6_b4_3': 23, '6_b5_1': 24}, 'enc_name': 'resnext50_32x4d', 'enc_dim': 2048, 'dec_dim': 1024, 'img_size1': 384, 'img_size2': 384, 'precision': 'amp', 'seed': 42, 'mix': 'cutmix', '

C:\Users\GunhaKim\anaconda3\envs\projectbase\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


total_steps:  57662
warmup_steps:  1149
learning_rate:  0.0
----- train, epoch1 -----


  0%|                                                                                         | 0/2306 [00:00<?, ?it/s]C:\Users\GunhaKim\anaconda3\envs\projectbase\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|                                                                               | 1/2306 [00:02<1:42:20,  2.66s/it]C:\Users\GunhaKim\anaconda3\envs\projectbase\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, 

RuntimeError: CUDA out of memory. Tried to allocate 1.17 GiB (GPU 0; 8.00 GiB total capacity; 3.56 GiB already allocated; 1.07 GiB free; 4.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF